In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import json
import pickle
import cv2
import matplotlib.pyplot as plt
import torch

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
trans_t = lambda t : torch.Tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1]]).float()

rot_phi = lambda phi : torch.Tensor([
    [1,0,0,0],
    [0,np.cos(phi),-np.sin(phi),0],
    [0,np.sin(phi), np.cos(phi),0],
    [0,0,0,1]]).float()

rot_theta = lambda th : torch.Tensor([
    [np.cos(th),0,-np.sin(th),0],
    [0,1,0,0],
    [np.sin(th),0, np.cos(th),0],
    [0,0,0,1]]).float()


def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi/180.*np.pi) @ c2w
    c2w = rot_theta(theta/180.*np.pi) @ c2w
    c2w = torch.Tensor(np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])) @ c2w
    return c2w

def read_pickle(pkl_path):
    with open(pkl_path, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        return u.load()

def batch_rodrigues(poses):
    """ poses: N x 3
    """
    batch_size = poses.shape[0]
    angle = np.linalg.norm(poses + 1e-8, axis=1, keepdims=True)
    rot_dir = poses / angle

    cos = np.cos(angle)[:, None]
    sin = np.sin(angle)[:, None]

    rx, ry, rz = np.split(rot_dir, 3, axis=1)
    zeros = np.zeros([batch_size, 1])
    K = np.concatenate([zeros, -rz, ry, rz, zeros, -rx, -ry, rx, zeros],
                       axis=1)
    K = K.reshape([batch_size, 3, 3])

    ident = np.eye(3)[None]
    rot_mat = ident + sin * K + (1 - cos) * np.matmul(K, K)

    return rot_mat

def get_rigid_transformation(rot_mats, joints, parents):
    """
    rot_mats: 24 x 3 x 3
    joints: 24 x 3
    parents: 24
    """
    # obtain the relative joints
    rel_joints = joints.copy()
    rel_joints[1:] -= joints[parents[1:]]

    # create the transformation matrix
    transforms_mat = np.concatenate([rot_mats, rel_joints[..., None]], axis=2)
    padding = np.zeros([24, 1, 4])
    padding[..., 3] = 1
    transforms_mat = np.concatenate([transforms_mat, padding], axis=1)

    # rotate each part
    transform_chain = [transforms_mat[0]]
    for i in range(1, parents.shape[0]):
        curr_res = np.dot(transform_chain[parents[i]], transforms_mat[i])
        transform_chain.append(curr_res)
    transforms = np.stack(transform_chain, axis=0)

    # obtain the rigid transformation
    padding = np.zeros([24, 1])
    joints_homogen = np.concatenate([joints, padding], axis=1)
    transformed_joints = np.sum(transforms * joints_homogen[:, None], axis=2)
    transforms[..., 3] = transforms[..., 3] - transformed_joints

    return transforms


def get_joints(coreview, index, smpl): # smpl
    param_path = os.path.join(coreview, "params", str(index)+".npy")

    params = np.load(param_path, allow_pickle=True).item()

    ### def get_transform_params(smpl, params)
    v_template = np.array(smpl['v_template'])
    shapedirs = np.array(smpl['shapedirs'])
    betas = params['shapes']
    v_shaped = v_template + np.sum(shapedirs * betas[None], axis=2) # v_shaped

    poses = params['poses'].reshape(-1, 3) # add pose blend shapes
    rot_mats = batch_rodrigues(poses) # 24 x 3 x 3
    joints = smpl['J_regressor'].dot(v_shaped)# obtain the joints

    # obtain the rigid transformation
    parents = smpl['kintree_table'][0]
    A = get_rigid_transformation(rot_mats, joints, parents)

    # apply global transformation
    R = cv2.Rodrigues(params['Rh'][0])[0]
    Th = params['Th']

    return joints, A, R, Th

def transform_joints(joints, A, Rh0, Th0):
    """ joints from smpl to world coordinate"""
    can_pts = (A[:, :3, :3] @ joints[..., None]).squeeze() + A[:,:3, 3] # joint transformation

    can_pts = (Rh0 @ can_pts.T) + Th0.T # smpl to world coordinate
    can_pts = can_pts.T

    return can_pts # joint position in world coordinate


# Remove 4 channels

In [17]:
import glob
from PIL import Image

imgs = []; poses = []
outputfolder = 'legoGT200c3'
# basedir="/mnt/disk1/JYChung/dataset/blender/blenderFiles2/LivingRoom13"
basedir="/mnt/disk1/JYChung/dataset/NeRF/blend_files/lego200"

os.makedirs("generated_data/"+outputfolder+"/image", exist_ok=True)
imglen = len(glob.glob(basedir+"/*.png"))
for i in range(imglen):
    img = plt.imread(os.path.join(basedir, "r_{0:03d}.png".format(i)))
    # plt.imsave("generated_data/"+outputfolder+"/image/r_{0:03d}.png".format(i), img[...,:-1]) # auto-save in 4 channel
    aa = Image.fromarray((img[...,:-1]*255).astype(np.uint8))
    aa.save("generated_data/"+outputfolder+"/image/r_{0:03d}.png".format(i)) # save in 3 channel


In [18]:
img2 = plt.imread(os.path.join("generated_data",outputfolder,"image", "r_{0:03d}.png".format(i)))
img2.shape

(200, 200, 3)

# 800SR 로 올린 다음에 GT 800 c4랑 결합하기

In [ ]:
import glob
from PIL import Image

imgs = []; poses = []
outputfolder = 'lego800SR4c4'
basedir="/home/chung/workspace/NeRF3DSR/generated_data/legoGT200c3/lego800SR4c3"
maskdir="/mnt/disk1/JYChung/dataset/NeRF/blend_files/lego800"

os.makedirs("generated_data/legoGT200c3/"+outputfolder, exist_ok=True)
imglen = len(glob.glob(basedir+"/*.png"))
for i in range(imglen):
    img = plt.imread(os.path.join(basedir, "r_{0:03d}_x4_SR.png".format(i)))
    mask = plt.imread(os.path.join(maskdir, "r_{0:03d}.png".format(i)))[...,-1,None]
    img = np.concatenate((img, mask), axis=-1)
    # plt.imsave("generated_data/"+outputfolder+"/image/r_{0:03d}.png".format(i), img[...,:-1]) # auto-save in 4 channel
    aa = Image.fromarray((img*255).astype(np.uint8))
    aa.save("generated_data/legoGT200c3/"+outputfolder+"/r_{0:03d}.png".format(i)) # save in 3 channel

# 기존 NeuS로 바꾸는 코드

In [11]:
import glob

imgs = []; poses = []
outputfolder = 'legoGT200c3'
# basedir="/mnt/disk1/JYChung/dataset/blender/blenderFiles2/LivingRoom13"
basedir="/mnt/disk1/JYChung/dataset/NeRF/blend_files/lego200"
with open(os.path.join(basedir, "transforms.json"), 'r') as file:
    meta = json.load(file)

imglen = len(glob.glob(basedir+"/*.png"))
for i in range(imglen):
    imgs.append(plt.imread(os.path.join(basedir, "r_{0:03d}.png".format(i))))
    poses.append(meta['frames'][i]['transform_matrix'])
poses = np.array(poses) # cam2world

H, W = imgs[0].shape[:2]
focal = .5 * W / np.tan(.5 * float(meta['camera_angle_x']))
K = np.zeros((3,3)); K[0,0]=focal; K[1,1]=focal; K[2,2]=1; K[0,2]=W/2; K[1,2]=H/2
# camera mat
camera_mat = np.zeros((4,4)); camera_mat[:3,:3]=K; camera_mat[3,3]=1 # c2w
scale_mat = np.eye(4); scale_mat[:3,:3] *= 2

camsph = {}
os.makedirs("generated_data/"+outputfolder+"/image", exist_ok=True)
os.makedirs("generated_data/"+outputfolder+"/mask", exist_ok=True)
for camnum in range(len(imgs)):
    # world_mat
    cam_coord_rot = np.array([[1,0,0],[0,-1,0],[0,0,-1]])
    pose = np.zeros((4,4))
    pose[:3,:3] = cam_coord_rot @ poses[camnum][:3,:3].T
    pose[:3,3] = -np.dot(cam_coord_rot @ poses[camnum][:3,:3].T, poses[camnum][:3,3].squeeze())
    pose[3,3] = 1
    world_mat = camera_mat @ pose
    # world_mat = camera_mat @ poses[camnum]
    
    # imgimg = (imgs[camnum][...,:3]*imgs[camnum][...,None,3] + 1.*(1.-imgs[camnum][...,None,3]))*255
    imgimg = imgs[camnum]*255
    cv2.imwrite("generated_data/"+outputfolder+"/image/{:03d}.png".format(camnum), cv2.cvtColor(imgimg.astype('uint8'), cv2.COLOR_BGR2RGB))
    cv2.imwrite("generated_data/"+outputfolder+"/mask/{:03d}.png".format(camnum), ((imgs[camnum][...,3]>0.5)*255.).astype('uint8'))

    # camsph['camera_mat_'+str(camnum)] = camera_mat
    camsph['world_mat_'+str(camnum)] = world_mat
    camsph['scale_mat_'+str(camnum)] = scale_mat
    

np.savez("generated_data/"+outputfolder+"/cameras_sphere.npz", **camsph)